<div align="center" dir="rtl">
<h2>تمرین چهارم درس بازیابی پیشرفته‌ی اطلاعات</h2>
<h3>گروه سی‌و‌سوم: اسامی</h3>
</div>
<div align="right" dir="rtl">
  <br>
    در این تمرین، تسک دسته‌بندی روی داده‌های گرد‌آوری شده از بوستان سعدی انجام می‌شود. ابتدا دیتاست مورد نیازمان از روی داده‌های خام که برای تمرین سوم جمع شده بود ساخته می‌شود، سپس قسمت پیش‌پردازش قرار دارد و در ادامه تسک روی داده‌ها انجام می‌شود.
</div>

<div align="right" dir="rtl">
    <h3>قسمت اول: آماده‌سازی دیتاست</h3>
  <br>
    در این قسمت فایل‌های داده‌های خام خوانده شده و دیتاست بیت به بیت برای کلاس‌بندی ایجاد می‌شود.
</div>

In [15]:
#import libraries for dataset generation
import pandas as pd
from os import listdir
!pip install hazm
!pip install torchtext
from hazm import *
import torch
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import itertools

You should consider upgrading via the 'C:\Users\ASUS\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


  Using cached torchtext-0.12.0-cp39-cp39-win_amd64.whl (1.5 MB)


You should consider upgrading via the 'C:\Users\ASUS\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [3]:
all_files = listdir('Boostan/')

i = 0
df = pd.DataFrame(columns=['beyt', 'baab'])

for p in all_files:
    f = open('Boostan/' + p, 'r', encoding='utf8')
    
    has_poems = True
    
    while has_poems:
        poem = ''
        
        while True:
            s = f.readline()
            if '$' in s:
                has_poems = False
                break
            elif '_' in s:
                break
            
            #age injaiim yani s ye mesra boode
            beyt = s + f.readline()[:-1]
            df.loc[i] = [beyt, p[:-4]]
            i = i + 1

In [16]:
print(df.at[2000, 'beyt'])
print(df.at[2000, 'baab'])
print(df.shape[0])

نپنداری قول معقول چو قانع شد سیم سنگ
باب ششم در قناعت
3845


<div align="right" dir="rtl">
    <h3>قسمت دوم: پیش‌پردازش</h3>
  <br>
    در این قسمت یک نمونه‌ی پیش‌پردازش شده از دیتاست تهیه می‌شود. دیتاست پیش‌پردازش‌نشده هم در کنار آن حفظ می‌شود که امکان مقایسه‌ی نتیجه روی دو دیتاست وجود داشته باشد.
    توضیحات دقیق این که هر گام از پیش‌پردازش شامل چه مواردی است، در تمرین سوم آورده شده است.
</div>

In [17]:
#import libraries for preprocessing
from __future__ import unicode_literals
from hazm import *
import json
import codecs

In [18]:
df_raw = df.copy()

<div align="right" dir="rtl">
    <h4>مرحله‌ی اول پیش‌پردازش</h4>
  <br>
    در این گام، برخی موارد در واژگان (مانند «گه» که همان «گاه» است) اصلاح می‌شوند. عملیات نرمالایز کردن و لمتایز کردن هم انجام داده می‌شود.
</div>

In [19]:
normalizer = Normalizer()
lemmatizer = Lemmatizer()

def preprocess_1(beyt):
    beyt = normalizer.normalize(beyt).replace(' ز ', ' از ').replace(' مر ', ' ') #normalize beshe va z beshe az
    
    words = word_tokenize(beyt)
    
    for j in range(0, len(words)):
        
        word = words[j]
        past_root = lemmatizer.lemmatize(word).split('#', 1)[0]
        past_root = past_root.replace('آ', 'ا')
        word_modified = word.replace(past_root, '')
        
        if word_modified != '':
            if word_modified[0] == 'ب' and len(word_modified) < 4:
                #yani fele maazi bode be forme ghadimi
                beyt = beyt.replace(word, word[1:])
        
        if word.endswith('گه'):
            beyt = beyt.replace(word, word[:-2]+'گاه')
        
        return beyt

for i in range(0, df.shape[0]):
    df.at[i, 'beyt'] = preprocess_1(df.at[i, 'beyt'])

<div align="right" dir="rtl">
    <h4>مرحله‌ی دوم پیش‌پردازش</h4>
  <br>
    در این گام، حالات باستانی افعال به حالت عادی تبدیل می‌شود تا افعال یک دست باشند.
</div>

In [20]:
def preprocess_2(beyt):
    words = word_tokenize(beyt)
    
    for j in range(0, len(words)):
        
        word = words[j]
        
        if word[0] == 'م':
            word_modified = 'ب' + word[1:]
            present_root = lemmatizer.lemmatize(word_modified).split('#', 1)[-1]
            present_root = present_root.replace('آ', 'ا')
            word_modified = word_modified.replace(present_root, '')
            
            if word_modified == 'ب':
                beyt = beyt.replace(word, 'ن'+word[1:])
    return beyt

for i in range(0, df.shape[0]):            
    df.at[i, 'beyt'] = preprocess_2(df.at[i, 'beyt'])

<div align="right" dir="rtl">
    <h4>مرحله‌ی سوم پیش‌پردازش</h4>
  <br>
    در این مرحله، استاپ ووردها حذف شده و برخی واژگان خاص پیش‌پردازش می‌شوند.
</div>

In [21]:
with open('Exception.txt', encoding='utf8') as exc:
    data = exc.read()
special_verbs = json.loads(data)

df_temp = pd.DataFrame(columns=['beyt', 'baab'])
stopwords = ['!','،','؟','ز','ار']+[normalizer.normalize(x.strip()) for x in codecs.open('stopwords.txt','r','utf-8').readlines()]


def preprocess_3(beyt):
    words = word_tokenize(beyt)
    words = [t for t in words if t not in stopwords]
    lemmatized_tokens = []
    
    for t in words:
        if t in special_verbs.keys():
            lemmatized_tokens.append(t)
        else:
            lemmatized_tokens.append(lemmatizer.lemmatize(t).split('#')[0])
    
    lemmatized = ' '.join(lemmatized_tokens)
            
    return lemmatized, lemmatized_tokens

for i in range(0, df.shape[0]):
    pp = preprocess_3(df.at[i, 'beyt'])
            
    df.at[i, 'beyt'] = pp[0]
    df_temp.at[i, 'beyt'] = pp[1]


In [22]:
print(df.at[2000, 'beyt'])
print(df.at[2000, 'baab'])

نپنداری قول معقول چو قانع سیم سنگ
باب ششم در قناعت


In [23]:
#test preprocessing
preprocess_3(preprocess_2(preprocess_1('''نپنداری این قول معقول نیست
چو قانع شدی سیم و سنگت یکی است''')))[0]

'نپنداری قول معقول چو قانع شد سیم سنگ'

<div align="right" dir="rtl">
    <h4>تابع پیش‌پردازش تک‌بیت</h4>
  <br>
    حال یک تابع درست می‌کنیم که یک بیت را پیش‌پردازش کند تا بعدا هنگام دریافت ورودی سامانه از آن استفاده کنیم.
</div>

In [24]:
def preprocess(beyt):
    return preprocess_3(preprocess_2(preprocess_1(beyt)))[0]

In [25]:
#test preprocessing functions
print(df_raw.at[2000, 'beyt'] + '\n----')
print(df.at[2000, 'beyt'] + '\n----')
print(preprocess(df_raw.at[2000, 'beyt']))

نپنداری قول معقول چو قانع شد سیم سنگ
----
نپنداری قول معقول چو قانع سیم سنگ
----
نپنداری قول معقول چو قانع سیم سنگ


<div align="right" dir="rtl">
    <h3>قسمت سوم: کلاس‌بندی با scikit</h3>
  <br>
    در این قسمت با ؟؟؟ کلاس‌بندی می‌کنیم
</div>

In [63]:
#import libraries for classification (1)
import os
import re
import shutil
import string
from collections import Counter
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from torch import nn
import torch.nn.functional as F
import time
from sklearn.metrics import f1_score
from sklearn import metrics
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset

In [53]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
corresponding_label = {"باب اول در عدل و تدبیر و رای":1,"باب دوم در احسان":2,"باب سوم در عشق و مستی و شور":3,"باب چهارم در تواضع":4,"باب پنجم در رضا":5,"باب ششم در قناعت":6,"باب هفتم در عالم تربیت":7,"باب هشتم در شکر بر عافیت":8,"باب نهم در توبه و راه صواب":9,'باب دهم در مناجات و ختم کتاب ':10}

In [54]:
#vectorized data
vectorizer = TfidfVectorizer(ngram_range=(1, 1), stop_words=None, norm='l2')
v = vectorizer.fit_transform(['نپنداری قول معقول چو قانع شد سیم سنگ', 'ممد ممد', 'ممد'])
#v = augment(v)

<div align=center>
		    <font size=4 color=green>
			    <br />
                مدل اول:
                </font>
                <font size=3>
                {اسم مدلتون رو اینجا بنویسید}
            	<br/>
			</font>
    <br/>
    </div>
    <hr/>

<div align=center>
		    <font size=4 color=green>
			    <br />
                مدل دوم:
                </font>
                <font size=3>
                {اسم مدلتون رو اینجا بنویسید}
            	<br/>
			</font>
    <br/>
    </div>
    <hr/>

<div align=center>
		    <font size=4 color=green>
			    <br />
                مدل سوم:
                </font>
                <font size=3>
                مدل شبکه عصبی استفاده از کتابخانه
                
torchtext
            	<br/>
			</font>
    <br/>
    </div>
    <hr/>

In [55]:
#split the data to training data and validation 
X_train, X_valid, Y_train, Y_valid= train_test_split(df['beyt'].tolist(),\
                                                      df['baab'].tolist(),\
                                                      test_size=0.2,\
                                                      stratify = df['baab'].tolist(),\
                                                      random_state=3)

train_dat =list(zip(Y_train,X_train))
valid_dat =list(zip(Y_valid,X_valid))

In [66]:
#define the voceb
train_iter = train_dat
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield word_tokenize(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [67]:
#define label and data piplines
text_pipeline = lambda x: vocab(word_tokenize(x))
label_pipeline = lambda x: corresponding_label[x]-1

In [68]:
#one layer model
class TextClassificationModel1(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel1, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

#two-layer model
class TextClassificationModel2(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel2, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc1 = nn.Linear(embed_dim,16)
        self.fc2 = nn.Linear(16,num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc1.weight.data.uniform_(-initrange, initrange)
        self.fc1.bias.data.zero_()
        self.fc2.weight.data.uniform_(-initrange, initrange)
        self.fc2.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        x = F.relu(self.fc1(embedded))
        x = self.fc2(x)
        return x

#three-layer model
class TextClassificationModel3(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc1 = nn.Linear(embed_dim,16)
        self.fc2 = nn.Linear(16,num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc1.weight.data.uniform_(-initrange, initrange)
        self.fc1.bias.data.zero_()
        self.fc2.weight.data.uniform_(-initrange, initrange)
        self.fc2.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        x = F.relu(self.fc1(embedded))
        x = self.fc2(x)
        return x

In [69]:
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

In [70]:
#embedding size is a hyperparameter which we should tune it
emsize = 32
train_iter1 = train_dat
num_class = len(set([label for (label, text) in train_iter1]))
vocab_size = len(vocab)
model = TextClassificationModel1(vocab_size, emsize, num_class).to(device)

In [71]:
#train the data
def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 50
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label = model(text, offsets)
        loss = criterion(predited_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            # print('| epoch {:3d} | {:5d}/{:5d} batches '
            #       '| accuracy {:8.3f}| loss {:8.3f}'.format(epoch, idx, len(dataloader),
            #                                   total_acc/total_count, loss.item()))
            total_acc, total_count = 0, 0
            start_time = time.time()

#evaluate the validation data
def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0
    y_true=[]
    y_pred=[]
    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            y_pred.append(predicted_label.argmax(1).tolist())
            y_true.append(label.tolist())
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    print(metrics.confusion_matrix(list(itertools.chain(*y_true)), list(itertools.chain(* y_pred)), labels=[0,1,2,3,4,5,6,7,8,9]))
    return total_acc/total_count, f1_score(list(itertools.chain(*y_true)),list(itertools.chain(* y_pred)), average='macro')

In [78]:
# Hyperparameters
EPOCHS = 5 # epoch
LR = 10 # learning rate
BATCH_SIZE = 32 # batch size for training

In [79]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None

train_iter2 = train_dat
valid_iter2= valid_dat


train_dataloader = DataLoader(train_iter2, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(valid_iter2, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)

In [80]:
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val,f_macro = evaluate(valid_dataloader)

    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} | f1_macro: {:8.3f}  '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val,f_macro))
    print('-' * 59)

[[146   5   2  25   0   0   1   0   3   0]
 [ 69   8   0  20   0   0   0   1   2   0]
 [ 53   1   1  16   0   0   2   0   0   0]
 [ 77   7   0  15   0   0   4   0   2   0]
 [ 32   2   0   7   0   0   0   0   0   0]
 [ 25   3   0   6   0   0   0   0   0   0]
 [ 59   1   1  23   0   0   3   0   2   0]
 [ 34   4   0  15   0   0   1   0   1   0]
 [ 46   0   1  14   0   0   0   0   6   0]
 [ 19   0   0   4   0   0   0   0   0   0]]
-----------------------------------------------------------
| end of epoch   1 | time:  0.48s | valid accuracy    0.233 | f1_macro:    0.087  
-----------------------------------------------------------
[[106  17   3  50   0   0   1   1   4   0]
 [ 38  19   3  31   0   0   0   0   9   0]
 [ 29   6   7  26   0   0   2   0   3   0]
 [ 33  13   4  50   0   0   2   0   3   0]
 [ 14   4   3  11   1   0   0   1   7   0]
 [ 14   2   2  13   0   1   0   0   2   0]
 [ 38   6   1  35   0   0   5   0   4   0]
 [ 15   6   2  24   1   0   1   1   5   0]
 [ 26   2   1  23   0 